In [1]:
from collections import Counter
import random
from statistics import variance
from typing import List, Set, Tuple, Union

In [2]:
class Gene:  # Match
    def __init__(self, team_1: Union[List, Set, Tuple], team_2: Union[List, Set, Tuple]):
        self.team_1 = set(team_1)
        self.team_2 = set(team_2)
        assert not self.team_1.intersection(self.team_2), "No overlapping players between teams"

        
    def __eq__(self, other):
        return ((self.team_1 == other.team_1) and (self.team_2 == other.team_2)
               or (self.team_1 == other.team_2) and (self.team_2 == other.team_1))
    
    def __hash__(self):
        return hash(f"{sorted(self.team_1)}.{sorted(self.team_2)}")
    
    def __str__(self):
        return f'{self.team_1} - {self.team_2}'
    
    def to_list(self):
        return list(self.team_1) + list(self.team_2)
        
# For simplicity, let's start with the case of a single Gene (match) played 
# simultaneously.
        
class Genome:  # Tournament
    def __init__(self, num_players, num_genes, genes: List['Gene']=None):
        self.num_players = num_players
        self.num_genes = num_genes
        if not genes:
            self.genes = [self._make_random_gene() for _ in range(self.num_genes)]
        else:
            self.genes = genes
    
    @property
    def fitness(self):
        # number of unique matches
        unique_matches = set()
        players = []
        for g in self.genes:
            unique_matches.add(g)
            players.extend(g.to_list())
        return len(unique_matches) * (1 - variance(Counter(players).values()))
        
    def _make_random_gene(self):
        num_players_for_gene = 4
        players = random.sample(range(self.num_players), num_players_for_gene)
        return Gene(players[:num_players_for_gene//2], players[num_players_for_gene//2:])
    
    def mutate(self):
        for i in range(len(self.genes)):
            if random.random() < 0.9:
                # TODO: do I want mutated gene to be based on original?
                self.genes[i] = self._make_random_gene()
    
    def __str__(self):
        return '\n'.join([str(g) for g in self.genes])

                
def crossover(a: 'Genome', b: 'Genome'):
    offspring_genes = []
    for left, right in zip(a.genes, b.genes):
        if random.random() < 0.5:
            offspring_genes.append(left)
        else:
            offspring_genes.append(right)
    return Genome(a.num_players, a.num_genes, offspring_genes)

def initial_population(size, num_players, num_genes):
    population = []
    for _ in range(size):
        population.append(Genome(num_players, num_genes))
    return population

def next_generation(current_generation):
    gen_by_fitness = sorted(current_generation, key=lambda g: g.fitness)
    # fittest from current generation will survive unmutated into future generation
    elite = gen_by_fitness[-1]  
    # all genomes from current generation that are in the top 40% based on fitness will reproduce
    breeding_population = set(gen_by_fitness[int(len(current_generation)*0.6):])
    
    children = [elite]
    while len(children) < len(current_generation):
        a = random.choice(list(breeding_population))
        b = random.choice(list(breeding_population - {a}))
        child = crossover(a, b)
        child.mutate()
        children.append(child)
    
    return children


In [3]:
%%time
num_players = 12
num_matches = 10
population = initial_population(100, num_players, num_matches)

elite = sorted(population, key=lambda g: g.fitness)[-1]
generation_elite_found = 0

for i in range(100):  # num generations
    population = next_generation(population)
    elite_in_generation = sorted(population, key=lambda g: g.fitness)[-1]
    if elite_in_generation.fitness > elite.fitness:
        elite = elite_in_generation
        generation_elite_found = i

print(generation_elite_found)
print(elite)
    

51
{0, 5} - {9, 4}
{8, 11} - {1, 10}
{6, 7} - {8, 10}
{0, 3} - {11, 7}
{9, 5} - {3, 6}
{4, 6} - {0, 10}
{3, 7} - {8, 5}
{1, 2} - {9, 4}
{8, 1} - {10, 2}
{0, 5} - {2, 11}
Wall time: 2.9 s


In [4]:
players = []
for g in elite.genes:
    players.extend(g.to_list())
Counter(players)

Counter({0: 4,
         5: 4,
         9: 3,
         4: 3,
         8: 4,
         11: 3,
         1: 3,
         10: 4,
         6: 3,
         7: 3,
         3: 3,
         2: 3})